In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import pickle

In [10]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [11]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan['duration'] = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb['duration'] = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df_jan.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [13]:
((df_jan.duration >= 1) & (df_jan.duration <= 60)).mean(), ((df_feb.duration >= 1) & (df_feb.duration <= 60)).mean()

(0.9812202822125979, 0.9800944077722545)

In [14]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [15]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [17]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []

df_jan[categorical] = df_jan[categorical].astype(str)
df_feb[categorical] = df_feb[categorical].astype(str)

In [18]:
dv = DictVectorizer() # turns dictionary to a vector
train_dicts = df_jan[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_feb[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
X_train.shape

(3009173, 515)

In [23]:
target = 'duration'
y_train = df_jan[target].values

linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_train)

RMSE = root_mean_squared_error(y_train, y_pred)
RMSE

7.649261927686161

In [24]:
target = 'duration'
y_train = df_jan[target].values
y_val = df_feb[target].values   

linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_val)

RMSE = root_mean_squared_error(y_val, y_pred)
RMSE

7.811817957524739